<a href="https://colab.research.google.com/github/okkuun/github-slideshow/blob/main/baseline_inKeras_single_regression_survive_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Nov  1 14:00:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [2]:
%%bash

mkdir -p ~/.kaggle
cp -f ./drive/MyDrive/kaggle/kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json
pip install -U kaggle
pip install wandb -qq
pip install tensorflow-addons

kaggle competitions download -c ventilator-pressure-prediction
unzip sample_submission.csv.zip
unzip test.csv.zip
unzip train.csv.zip




Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               


100%|##########| 8.50M/8.50M [00:00<00:00, 77.9MB/s]
100%|##########| 139M/139M [00:01<00:00, 108MB/s] 
100%|##########| 75.4M/75.4M [00:00<00:00, 127MB/s]


In [3]:
import numpy as np
import pandas as pd

# import optuna

import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/274717

import tensorflow as tf
from tensorflow import keras
# from tensorflow.keras import layers
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize, QuantileTransformer
from sklearn.model_selection import train_test_split, GroupKFold, KFold

import wandb
from wandb.keras import WandbCallback
import yaml
import gc

In [4]:
# making config

cfg = yaml.safe_load(
    """
    globals:
        name: keras_single_11
        seed: 2021

    features:
        all_features_cat: []
        all_features_con: []
        all_features: []

    model:
        hidden_size: 1024
        epochs: 300
        drop_rate: 0.2
        batch_size: 1024

    optimizer:
        lr: 1.0e-3
        weight_decay: 1.0e-6
        max_grad_norm: 1000
        min_lr: 1.0e-5
        scheduler: ReduceLROnPlateau
    """
)

In [5]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    gc.collect()
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [7]:
# [TODO] 特徴量の限定（過学習の一因）
# [TODO] 二乗を混ぜるように工夫
def add_features(df):
    df['u_out_formodel'] = df['u_out'].copy()
    df['cross']= df['u_in'] * df['u_out']
    df['cross2']= df['time_step'] * df['u_out']
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    df['time_step_cumsum'] = df.groupby(['breath_id'])['time_step'].cumsum()
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    print("Step-1...Completed")
    
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
    df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
    df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
    df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
    df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
    df = df.fillna(0)
    print("Step-2...Completed")
    
    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['breath_id__u_in__mean'] = df.groupby(['breath_id'])['u_in'].transform('mean')
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    print("Step-3...Completed")
    
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
    df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
    print("Step-4...Completed")
    
    df['one'] = 1
    df['count'] = (df['one']).groupby(df['breath_id']).cumsum()
    df['u_in_cummean'] =df['u_in_cumsum'] /df['count']
    
    df['breath_id_lag']=df['breath_id'].shift(1).fillna(0)
    df['breath_id_lag2']=df['breath_id'].shift(2).fillna(0)
    df['breath_id_lagsame']=np.select([df['breath_id_lag']==df['breath_id']],[1],0)
    df['breath_id_lag2same']=np.select([df['breath_id_lag2']==df['breath_id']],[1],0)
    df['breath_id__u_in_lag'] = df['u_in'].shift(1).fillna(0)
    df['breath_id__u_in_lag'] = df['breath_id__u_in_lag'] * df['breath_id_lagsame']
    df['breath_id__u_in_lag2'] = df['u_in'].shift(2).fillna(0)
    df['breath_id__u_in_lag2'] = df['breath_id__u_in_lag2'] * df['breath_id_lag2same']
    print("Step-5...Completed")
    
    _delta_time = df.groupby('breath_id')['time_step'].diff().fillna(0)
    for i_shift in range(1, 1+1):
        df[f'delta_u_in_{i_shift}'] = df[f'u_in_diff{i_shift}'] / np.where(_delta_time>=0.03*i_shift, _delta_time, 0.03*i_shift)

    df['area_gap'] = df['u_in'] * _delta_time
    df['area_cumsum_gap'] = (df['area_gap']).groupby(df['breath_id']).cumsum()

    df['ewm_u_in_mean'] = (df\
                           .groupby('breath_id')['u_in']\
                           .apply(lambda x: x.ewm(halflife=9)\
                           .mean())\
                           .reset_index(level=0,drop=True))
    df[["15_in_sum","15_in_min","15_in_max","15_in_mean"]] = (df\
                                                              .groupby('breath_id')['u_in']\
                                                              .apply(lambda x: x.rolling(window=15,min_periods=1)\
                                                              .agg({"15_in_sum":"sum",
                                                                    "15_in_min":"min",
                                                                    "15_in_max":"max",
                                                                    "15_in_mean":"mean"}))\
                                                               .reset_index(level=0,drop=True))
    print("Step-6...Completed")

    # df = reduce_mem_usage(df)
    
    df['u_in_lagback_diff1'] = df['u_in'] - df['u_in_lag_back1']
    df['u_out_lagback_diff1'] = df['u_out'] - df['u_out_lag_back1']
    df['u_in_lagback_diff2'] = df['u_in'] - df['u_in_lag_back2']
    df['u_out_lagback_diff2'] = df['u_out'] - df['u_out_lag_back2']
    print("Step-7...Completed")

    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df = pd.get_dummies(df)
    print("Step-8...Completed")

    for c in df.columns:
        if c=="u_out":
            df[c] = df[c].astype("uint8")
        elif df[c].dtype=="float64":
            df[c] = df[c].astype("float32")
    return df

In [10]:

# なるべくu_inではなくcrossを使う
# [TODO] u_inやu_out系の特徴量もあった方が精度良いか？
def add_features_2(df):
    df['u_out_formodel'] = df['u_out'].copy()
    df['cross']= df['u_in'] * (1 - df['u_out'])
    df['cross2']= df['time_step'] * (1 - df['u_out'])
    # df['area'] = df['time_step'] * df['u_in']
    # df['area'] = df.groupby('breath_id')['area'].cumsum()
    df['area2'] = df['time_step'] * df['cross']
    df['area2'] = df.groupby('breath_id')['area2'].cumsum()
    # df['time_step_cumsum'] = df.groupby(['breath_id'])['time_step'].cumsum()
    df['cross2_cumsum'] = df.groupby(['breath_id'])['cross2'].cumsum()
    # df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    df['cross_cumsum'] = (df['cross']).groupby(df['breath_id']).cumsum()
    print("Step-1...Completed")
    
    df['cross_lag1'] = df.groupby('breath_id')['cross'].shift(1)
    # df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
    df['cross_lag_back1'] = df.groupby('breath_id')['cross'].shift(-1)
    # df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
    df['cross_lag2'] = df.groupby('breath_id')['cross'].shift(2)
    # df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
    df['cross_lag_back2'] = df.groupby('breath_id')['cross'].shift(-2)
    # df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
    df['cross_lag3'] = df.groupby('breath_id')['cross'].shift(3)
    # df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
    df['cross_lag_back3'] = df.groupby('breath_id')['cross'].shift(-3)
    # df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
    df['cross_lag4'] = df.groupby('breath_id')['cross'].shift(4)
    # df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
    df['cross_lag_back4'] = df.groupby('breath_id')['cross'].shift(-4)
    # df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
    df = df.fillna(0)
    print("Step-2...Completed")
    
    # df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    # df['breath_id__u_in__mean'] = df.groupby(['breath_id'])['u_in'].transform('mean')
    # df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    # df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    df['breath_id__cross__max'] = df.groupby(['breath_id'])['cross'].transform('max')
    df['breath_id__cross__mean'] = df.groupby(['breath_id'])['cross'].transform('mean')
    df['breath_id__cross__diffmax'] = df.groupby(['breath_id'])['cross'].transform('max') - df['cross']
    df['breath_id__cross__diffmean'] = df.groupby(['breath_id'])['cross'].transform('mean') - df['cross']
    print("Step-3...Completed")
    
    # df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    # df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
    # df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    # df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
    # df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    # df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    # df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    # df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
    df['cross_diff1'] = df['cross'] - df['cross_lag1']
    df['cross_diff2'] = df['cross'] - df['cross_lag2']
    df['cross_diff3'] = df['cross'] - df['cross_lag3']
    df['cross_diff4'] = df['cross'] - df['cross_lag4']
    print("Step-4...Completed")
    
    df['one'] = 1
    df['count'] = (df['one']).groupby(df['breath_id']).cumsum()
    # df['u_in_cummean'] =df['u_in_cumsum'] /df['count']
    df['cross_cummean'] = df['cross_cumsum'] / df['count']
    
    df['breath_id_lag']=df['breath_id'].shift(1).fillna(0)
    df['breath_id_lag2']=df['breath_id'].shift(2).fillna(0)
    df['breath_id_lagsame']=np.select([df['breath_id_lag']==df['breath_id']],[1],0)
    df['breath_id_lag2same']=np.select([df['breath_id_lag2']==df['breath_id']],[1],0)
    # df['breath_id__u_in_lag'] = df['u_in'].shift(1).fillna(0)
    # df['breath_id__u_in_lag'] = df['breath_id__u_in_lag'] * df['breath_id_lagsame']
    # df['breath_id__u_in_lag2'] = df['u_in'].shift(2).fillna(0)
    # df['breath_id__u_in_lag2'] = df['breath_id__u_in_lag2'] * df['breath_id_lag2same']
    df['breath_id__cross_lag'] = df['cross'].shift(1).fillna(0)
    df['breath_id__cross_lag'] = df['breath_id__cross_lag'] * df['breath_id_lagsame']
    df['breath_id__cross_lag2'] = df['cross'].shift(2).fillna(0)
    df['breath_id__cross_lag2'] = df['breath_id__cross_lag2'] * df['breath_id_lag2same']
    print("Step-5...Completed")
    
    _delta_time = df.groupby('breath_id')['time_step'].diff().fillna(0)
    for i_shift in range(1, 4+1):
        # df[f'delta_u_in_{i_shift}'] = df[f'u_in_diff{i_shift}'] / np.where(_delta_time>=0.03*i_shift, _delta_time, 0.03*i_shift)
        df[f'delta_cross_{i_shift}'] = df[f'cross_diff{i_shift}'] / np.where(_delta_time>=0.03*i_shift, _delta_time, 0.03*i_shift)

    # df['ewm_u_in_mean'] = (df\
    #                        .groupby('breath_id')['u_in']\
    #                        .apply(lambda x: x.ewm(halflife=9)\
    #                        .mean())\
    #                        .reset_index(level=0,drop=True))
    # df[["15_in_sum","15_in_min","15_in_max","15_in_mean"]] = (df\
    #                                                           .groupby('breath_id')['u_in']\
    #                                                           .apply(lambda x: x.rolling(window=15,min_periods=1)\
    #                                                           .agg({"15_in_sum":"sum",
    #                                                                 "15_in_min":"min",
    #                                                                 "15_in_max":"max",
    #                                                                 "15_in_mean":"mean"}))\
    #                                                            .reset_index(level=0,drop=True))
    df['ewm_cross_mean'] = (df\
                           .groupby('breath_id')['cross']\
                           .apply(lambda x: x.ewm(halflife=9)\
                           .mean())\
                           .reset_index(level=0,drop=True))
    df[["15_cross_sum","15_cross_min","15_cross_max","15_cross_mean"]] = (df\
                                                              .groupby('breath_id')['cross']\
                                                              .apply(lambda x: x.rolling(window=10,min_periods=1)\
                                                              .agg({"15_cross_sum":"sum",
                                                                    "15_cross_min":"min",
                                                                    "15_cross_max":"max",
                                                                    "15_cross_mean":"mean"}))\
                                                               .reset_index(level=0,drop=True))
    print("Step-6...Completed")
    
    # df['u_in_lagback_diff1'] = df['u_in'] - df['u_in_lag_back1']
    # df['u_out_lagback_diff1'] = df['u_out'] - df['u_out_lag_back1']
    # df['u_in_lagback_diff2'] = df['u_in'] - df['u_in_lag_back2']
    # df['u_out_lagback_diff2'] = df['u_out'] - df['u_out_lag_back2']
    df['cross_lagback_diff1'] = df['cross'] - df['cross_lag_back1']
    df['cross_lagback_diff2'] = df['cross'] - df['cross_lag_back2']
    print("Step-7...Completed")

    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df = pd.get_dummies(df)
    print("Step-8...Completed")

    for c in df.columns:
        if c=="u_out":
            df[c] = df[c].astype("uint8")
        elif df[c].dtype=="float64":
            df[c] = df[c].astype("float32")
    return df

In [6]:
# train = pd.read_csv('./train.csv')
# test = pd.read_csv('./test.csv')
# sample_sub = pd.read_csv('./sample_submission.csv')

train = pd.read_csv('./drive/MyDrive/vpp/train2_pre.csv', index_col=0)
test = pd.read_csv('./drive/MyDrive/vpp/test2_pre.csv', index_col=0)
sample_sub = pd.read_csv('./sample_submission.csv')

# # drop
# train = train.drop(columns=drop_features)
# test = test.drop(columns=drop_features)

# u_inを非線形変換してみる
# train['u_in'] = np.log1p(np.where(train['u_in'].values>0, train['u_in'].values, 0))
# test['u_in'] = np.log1p(np.where(test['u_in'].values>0, test['u_in'].values, 0))

# train = add_features_2(train)
# # train = reduce_mem_usage(train)
# test = add_features_2(test)
# # test = reduce_mem_usage(test)

# train.drop(['one','count',
#             'breath_id_lag','breath_id_lag2','breath_id_lagsame',
#             'breath_id_lag2same'], axis=1, inplace=True)

# test = test.drop(['one','count', 'breath_id_lag',
#                   'breath_id_lag2','breath_id_lagsame',
#                   'breath_id_lag2same'], axis=1)

targets = train[['pressure']].to_numpy().reshape(-1, 80)
u_outs = train[['u_out']].to_numpy().reshape(-1, 80)

feature_list = train.columns.tolist()
for drop_c in ['pressure', 'id', 'breath_id']:
    feature_list.remove(drop_c)

feature_list.remove('u_out')
cfg['features']['all_features'] = feature_list.copy()

# 数値変数とカテゴリ変数で分けて変換
feature_list_con = feature_list.copy()
feature_list_cat = []
for col in feature_list:
    if len(train[col].unique()) <= 10:
        feature_list_con.remove(col)
        feature_list_cat.append(col)

cfg['features']['all_features_con'] = feature_list_con
cfg['features']['all_features_cat'] = feature_list_cat

# QS = RobustScaler()
# train.loc[:, feature_list_con] = QS.fit_transform(train.loc[:, feature_list_con])
# test.loc[:, feature_list_con] = QS.transform(test.loc[:, feature_list_con])

# # u_in系の特徴量にlogを適用する
# for col in feature_list_con:
#     str_list = col.split('_')
    
#     if 'u' in str_list and 'in' in str_list:
#         train[col] = np.log1p(train[col])
#         test[col] = np.log1p(test[col])

#     elif 'cross' in str_list:
#         train[col] = np.log1p(train[col])
#         test[col] = np.log1p(test[col])

# 数値変数の２乗を混ぜる
# for col in feature_list:
#     train[col+'_x2'] = train[col]**2
#     test[col+'_x2'] = test[col]**2

# memory reduction
# train = reduce_mem_usage(train)
# test = reduce_mem_usage(test)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
all_pressure = sorted(train.pressure.unique())
# all_pressure_torch = torch.from_numpy(train.pressure.values).clone()
P_MIN = np.min(all_pressure)
P_MAX = np.max(all_pressure)
P_STEP = all_pressure[1] - all_pressure[0]

In [8]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    BATCH_SIZE = strategy.num_replicas_in_sync * 128
    print("Running on TPU:", tpu.master())
    print(f"Batch Size: {BATCH_SIZE}")
    
except ValueError:
    strategy = tf.distribute.get_strategy()
    BATCH_SIZE = 512
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    print(f"Batch Size: {BATCH_SIZE}")

INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.106.9.210:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.106.9.210:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU: grpc://10.106.9.210:8470
Batch Size: 1024


In [11]:
# from tensorflow.keras.layers import Bidirectional, LSTM
# from tensorflow.keras.layers import Dense, Dropout, Input, LayerNormalization
# from tensorflow.keras.layers import Concatenate, Add, GRU

# def dnn_model():
    
#     x_input = Input(shape=(80, len(cfg['features']['all_features'])))
#     x_emb = Dense(units=64, activation='selu',kernel_initializer=tf.keras.initializers.lecun_normal(seed=2021))(x_input)
#     x_emb = LayerNormalization(axis=-1)(x_emb)
    
#     x1 = Bidirectional(LSTM(units=768, return_sequences=True))(x_emb)
#     x2 = Bidirectional(LSTM(units=512, return_sequences=True))(x1)
#     x3 = Bidirectional(LSTM(units=256, return_sequences=True))(x2)
    
#     z2 = Bidirectional(GRU(units=256, return_sequences=True))(x2)
#     z3 = Bidirectional(GRU(units=128, return_sequences=True))(Add()([x3, z2]))
    
#     x = Concatenate(axis=2)([x3, z2, z3])
#     x = Bidirectional(LSTM(units=192, return_sequences=True))(x)
    
#     x = Dense(units=128, activation='selu',kernel_initializer=tf.keras.initializers.lecun_normal(seed=2021))(x)
#     x = LayerNormalization()(x)
    
#     x_output = Dense(units=1,kernel_initializer=tf.keras.initializers.lecun_normal(seed=2021))(x)

#     model = tf.keras.models.Model(inputs=x_input, outputs=x_output, name='DNN_Model')
#     return model

# model = dnn_model()
# model.summary()

In [9]:
from tensorflow.keras import layers as L
from tensorflow.keras.layers import Bidirectional, LSTM, GRU
from tensorflow.keras.layers import Dense, Dropout, Input, LayerNormalization
from tensorflow.keras.layers import Concatenate, Add, Multiply, BatchNormalization
# # [TODO] 分類問題への発展（別ファイル？）
def dnn_model():
    x_input = Input(shape=(80, len(cfg['features']['all_features'])))
    x_emb = Dense(units=64, activation='selu',kernel_initializer=tf.keras.initializers.lecun_normal(seed=2021))(x_input)
    x_emb = LayerNormalization(axis=1)(x_emb)
    
    x1 = Bidirectional(LSTM(units=768, return_sequences=True))(x_input)
    x2 = Bidirectional(LSTM(units=512, return_sequences=True))(x1)
    x3 = Bidirectional(LSTM(units=384, return_sequences=True))(x2)
    x4 = Bidirectional(LSTM(units=256, return_sequences=True))(x3)
    x5 = Bidirectional(LSTM(units=128, return_sequences=True))(x4)
    
    z2 = Bidirectional(GRU(units=384, return_sequences=True))(x2)
    
    z31 = Multiply()([x3, z2])
    z31 = BatchNormalization()(z31)
    z3 = Bidirectional(GRU(units=256, return_sequences=True))(z31)
    
    z41 = Multiply()([x4, z3])
    z41 = BatchNormalization()(z41)
    z4 = Bidirectional(GRU(units=128, return_sequences=True))(z41)
    
    z51 = Multiply()([x5, z4])
    z51 = BatchNormalization()(z51)
    z5 = Bidirectional(GRU(units=64, return_sequences=True))(z51)
    
    x = Concatenate(axis=2)([x5, z2, z3, z4, z5])
    
    x = Dense(
        units=128, activation='selu', 
        kernel_initializer=tf.keras.initializers.lecun_normal(seed=2021)
    )(x)
    x = LayerNormalization()(x)

    x_output = Dense(units=1)(x)

    model = tf.keras.models.Model(inputs=x_input, outputs=x_output, 
                name='DNN_Model')
    return model

model = dnn_model()
model.summary()

Model: "DNN_Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 80, 53)]     0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 80, 1536)     5050368     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 80, 1024)     8392704     bidirectional[0][0]              
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 80, 768)      4328448     bidirectional_1[0][0]            
__________________________________________________________________________________________

In [13]:
# from tensorflow.keras import layers as L
# from tensorflow.keras.layers import Bidirectional, LSTM, GRU
# from tensorflow.keras.layers import Dense, Dropout, Input, LayerNormalization
# from tensorflow.keras.layers import Concatenate, Add, Multiply, BatchNormalization
# def dnn_model(strategy):   
#     with strategy.scope():
#         inp = L.Input(shape=(80, len(cfg['features']['all_features'])),name="Input")

#         emb = Dense(units=64, activation='selu', kernel_initializer=tf.keras.initializers.lecun_normal(seed=2021))(inp)
#         emb = LayerNormalization(axis=-1)(emb)

#         x1 = L.Bidirectional(L.LSTM(units=1024, return_sequences=True),name="BiLSTM1")(emb)
#         x2 = L.Bidirectional(L.LSTM(units=512, return_sequences=True),name="BiLSTM2")(x1)
#         x3 = L.Bidirectional(L.LSTM(units=256, return_sequences=True),name="BiLSTM3")(x2)
#         x4 = L.Bidirectional(L.LSTM(units=128, return_sequences=True),name="BiLSTM4")(x3)

#         z2 = L.Bidirectional(L.GRU(units=256, return_sequences=True),name="BiGRU1")(x2)
#         z3 = L.Bidirectional(L.GRU(units=128, return_sequences=True),name="BiGRU2")(L.Add()([x3, z2]))
#         z4 = L.Bidirectional(L.GRU(units=128, return_sequences=True),name="BiGRU3")(L.Add()([x4, z3]))

#         x = L.Concatenate(axis=2, name="Concat")([x4, z2, z3, z4])

#         out = L.Dense(128, activation='selu', kernel_initializer=tf.keras.initializers.lecun_normal(seed=2021))(x)
#         out = LayerNormalization(axis=-1)(out)
#         out = L.Dense(1)(out)

#         model = tf.keras.Model(inputs=inp, outputs=out)
    
#     return model

# model = dnn_model(strategy)
# model.summary()

In [14]:
strategy

In [10]:
# wandbを起動
# wandb.finish()
run = wandb.init(
    project='kaggle-vpp',
    name=cfg['globals']['name'],
    config=cfg
)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [16]:


# cross-validation(group k-fold by breath_id)
# kf = KFold(n_splits=5, shuffle=True, random_state=cfg['globals']['seed'])
# fold_dict = {'breath_id': [], 'fold': []}
# for i, (train_breath, val_breath) in enumerate(kf.split(train['breath_id'].unique())):
#     fold_dict['breath_id'] += val_breath.tolist()
#     fold_dict['fold'] += np.full(len(val_breath), i).tolist()
# fold_df = pd.DataFrame(fold_dict.values(), index=fold_dict.keys()).T
# train = pd.merge(train, fold_df, on='breath_id')
# train['fold'] = train['fold'].astype(int)

In [17]:
def GBVPP_loss(y_true, y_pred, cols = 80):
    u_out = y_true[:, cols: ]
    y = y_true[:, :cols ]

    w = 1 - u_out
    mae = w * tf.abs(y - y_pred)
    return tf.reduce_sum(mae, axis=-1) / tf.reduce_sum(w, axis=-1)

In [ ]:
def smooth_l1_loss(y_true, y_pred, beta=0.1):
    # y = y_true[:, :80]

    error = y - y_pred
    mode = tf.keras.backend.abs(error) < beta
    squared_loss = 0.5 * tf.keras.backend.square(error) / beta
    linear_loss = tf.keras.backend.abs(error) - 0.5 * beta
    return tf.where(mode, squared_loss, linear_loss)

In [ ]:
# y_true \in [y_true, u_out]で構成
# y_pred \in [pressure_in, pressure_out]で構成
# [TODO] beta = 0.005, 0.001
def mask_smooth_l1_loss(y_true, y_pred, cols=80):
    u_out = y_true[:, cols: ]
    # u_out = u_out == 0
    pressure_true = y_true[:, :cols ]

    pressure_in = y_pred[:, :cols]
    pressure_out = y_pred[:, cols:]
    
    loss = smooth_l1_loss(pressure_true[u_out==0], pressure_in[u_out==0]) * 2.

    return loss


In [ ]:
def mask_l1_loss(y_true, y_pred, cols=80):
    u_out = y_true[:, cols:]
    pressure_true = y_true[:, :cols]

    return tf.keras.losses.mean_absolute_error(pressure_true[u_out == 0], y_pred[u_out == 0])

In [ ]:
def mask_l1_metrics(y_true, y_pred, cols=80):
    u_out = y_true[:, cols:]
    pressure_true = y_true[:, :cols]

    return tf.keras.metrics.mean_absolute_error(pressure_true[u_out == 0], y_pred[u_out == 0])

In [ ]:
def all_l1_loss(y_true, y_pred):
    pressure_true = y_true[:, :80]
    return tf.keras.losses.mean_absolute_error(pressure_true, y_pred)

In [ ]:
# main
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
# detect and init the TPU

train_data = train[cfg['features']['all_features']].values.reshape(-1, 80, len(cfg['features']['all_features']))
test_data = test[cfg['features']['all_features']].values.reshape(-1, 80, len(cfg['features']['all_features']))

# gpu_strategy = tf.distribute.get_strategy()
test_preds = []
features = cfg['features']['all_features']
kf = KFold(n_splits=16, shuffle=True, random_state=2021)
for fold, (train_idx, valid_idx) in enumerate(kf.split(train_data, targets)):
    # if fold != 0:
    #     continue
    K.clear_session()
    X_train, X_valid = train_data[train_idx], train_data[valid_idx]
    y_train, y_valid = targets[train_idx], targets[valid_idx]
    u_out_train, u_out_valid = u_outs[train_idx], u_outs[valid_idx]

    # y_train_log, y_valid_log = np.log1p(y_train), np.log1p(y_valid)

    # y_train = np.concatenate([y_train, u_out_train], axis=1)
    # y_valid = np.concatenate([y_valid, u_out_valid], axis=1)
    
    # model
    with strategy.scope():
        model = dnn_model()
        lr = tf.keras.experimental.CosineDecayRestarts(1e-3, first_decay_steps=100 * X_train.shape[0] // 2056, t_mul=1.0, alpha=1e-3)
    # lr = 1e-3
        opt = tf.optimizers.Adam(learning_rate=lr)
        # opt = tfa.optimizers.AdamW(learning_rate=lr, weight_decay=1e-7)
        # loss_fn = tf.keras.losses.MeanAbsoluteError()
        # acc_fn = GBVPP_loss()
        # loss_fn = smooth_l1_loss
        model.compile(optimizer=opt, loss='mae')

    # lr = ReduceLROnPlateau(monitor="val_loss", factor=0.85, patience=7, verbose=1)
#         lr = WarmupExponentialDecay(lr_base=1e-3, decay=1e-5, warmup_epochs=30)
    es = EarlyStopping(monitor="val_loss", patience=95, verbose=1, mode="min", restore_best_weights=True)
#         masked_mae = MaskedValidationMAE()
    output_path = f"./drive/MyDrive/vpp/{cfg['globals']['name']}/"
    checkpoint_filepath = output_path + f"folds{fold}.hdf5"
    sv = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
        save_weights_only=False, mode='auto', save_freq='epoch', options=None
    )

    # model.fit(
    #     X_train, np.append(y_train, u_out_train, axis=1),
    #     validation_data=(X_valid, np.append(y_valid, u_out_valid, axis=1)),
    #     epochs=cfg['model']['epochs'], batch_size=cfg['model']['batch_size'],
    #     callbacks=[lr, es, sv, WandbCallback()]
    # )

    histry = model.fit(
        X_train, y_train,
        validation_data=(X_valid, y_valid),
        epochs=500, batch_size=BATCH_SIZE,
        callbacks=[es, sv], shuffle=True,
        steps_per_epoch=X_train.shape[0] // 2056, validation_steps = X_valid.shape[0] // BATCH_SIZE
    )

    with strategy.scope():
        model = keras.models.load_model(checkpoint_filepath)

    # validationで結果見る
    pred_pressure = model.predict(X_valid, batch_size=BATCH_SIZE)
    valid_mask = u_out_valid == 0
    score = mae(y_valid[:, :80][valid_mask], pred_pressure.reshape(-1, 80)[valid_mask])
    print(f"Fold-{fold} | OOF Score: {score}")

    # test
    pred_pressure = model.predict(test_data, batch_size=BATCH_SIZE, verbose=2).squeeze().reshape(-1, 1).squeeze()
    test_preds.append(pred_pressure)

    del model, X_train, X_valid, pred_pressure, y_train, y_valid
    gc.collect()

In [ ]:
sample_sub["pressure"] = np.median(np.vstack(test_preds),axis=0)
sample_sub["pressure"] = np.round((sample_sub.pressure - P_MIN)/P_STEP) * P_STEP + P_MIN
sample_sub["pressure"] = np.clip(sample_sub.pressure, P_MIN, P_MAX)
sample_sub.to_csv(output_path + 'submission.csv', index=False)

# validationで実測値と予測値を観察

In [ ]:
import tensorflow_addons as tfa
from tensorflow_addons.rnn import LayerNormLSTMCell
# detect and init the TPU

# with tpu_strategy.scope():

# gpu_strategy = tf.distribute.get_strategy()
fold_observe = 2
features = cfg['features']['all_features']
kf = KFold(n_splits=7, shuffle=True, random_state=2021)
for fold, (train_idx, valid_idx) in enumerate(kf.split(train_data, targets)):
    if fold != fold_observe:
        continue
    X_train, X_valid = train_data[train_idx], train_data[valid_idx]
    y_train, y_valid = targets[train_idx], targets[valid_idx]
    u_out_train, u_out_valid = u_outs[train_idx], u_outs[valid_idx]
    
    # model
    for fold_pred in range(7):
        if fold_pred == fold_observe:
            continue
        valid_preds = []
        output_path = f"./drive/MyDrive/vpp/{cfg['globals']['name']}/"
        checkpoint_filepath = output_path + f"folds{fold_pred}.hdf5"
        with tpu_strategy.scope():
            model = keras.models.load_model(checkpoint_filepath)

            valid_preds.append(model.predict(X_valid, batch_size=2048, verbose=2).squeeze().reshape(-1, 1).squeeze())

        del model
    valid_preds = np.median(np.vstack(valid_preds), axis=0)
    valid_preds = np.round((valid_preds - P_MIN) / P_STEP) * P_STEP + P_MIN
    valid_preds = np.clip(valid_preds, P_MIN, P_MAX)
    if fold == fold_observe:
        break
    gc.collect()

In [ ]:
valid_idx.shape

In [ ]:
u_out_valid.shape

In [ ]:
valid_preds.reshape(-1, 80).shape

In [ ]:
train['breath_id'].unique()[:30]

In [ ]:
valid_preds = valid_preds.reshape(-1, 80)
breath_id_list = train['breath_id'].unique()[valid_idx].tolist()
mae_list = []
for i, a_breath_id in enumerate(breath_id_list):
    is_u_out = u_out_valid[i] == 0
    mae_loss = mae(y_valid[i, is_u_out], valid_preds[i, is_u_out])
    mae_list.append(mae_loss)

rank_mae_valid = pd.DataFrame()
rank_mae_valid['breath_id'] = breath_id_list
rank_mae_valid['mae'] = mae_list

In [ ]:
rank_mae_valid = rank_mae_valid.sort_values('mae')
y_valid = y_valid[rank_mae_valid.index]
valid_preds = valid_preds[rank_mae_valid.index]
rank_mae_valid.head()

In [ ]:
rank_mae_valid.tail()

In [ ]:
rank_mae_valid['mae'].mean()

In [ ]:
y_valid

In [ ]:
y_valid[rank_mae_valid.index]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def plot_one_breath_id(top=True, rank=1, show_data=False):
    """
    top: Trueで上位○位、Falseで下位○位
    rank: 順位
    show_data: はじめのいくつかの特徴量を表示
    """
    if top:
        rank -= 1
    else:
        rank *= -1
    a_rank_mae = rank_mae_valid.iloc[rank]
    print(f"breath_id : {a_rank_mae['breath_id']}")
    print(f"MAE : {a_rank_mae['mae']}")

    a_breath_id_data = train[train['breath_id']==a_rank_mae['breath_id']]

    # u_outが切り替わった瞬間をキャッチ
    idx_change_u_out = a_breath_id_data['u_out'] - a_breath_id_data['u_out'].shift(1).fillna(0)
    idx_change_u_out = np.arange(80)[idx_change_u_out!=0]
    print(idx_change_u_out)

    # display(a_breath_id)
    plt.figure()
    plt.plot(np.arange(len(a_breath_id_data)), y_valid[rank], label='True')
    plt.plot(np.arange(len(a_breath_id_data)), valid_preds[rank], label='Pred')
    plt.axvline(idx_change_u_out, ymax=40, color='black', linestyle='--', label='Change u_out')
    plt.grid()
    plt.legend()
    plt.show()
    plt.close()

    if show_data:
        display(a_breath_id_data.head())

In [ ]:
plot_one_breath_id(rank=1)

In [ ]:
plot_one_breath_id(rank=2)

In [ ]:
plot_one_breath_id(rank=10)

In [ ]:
plot_one_breath_id(rank=500)

In [ ]:
plot_one_breath_id(top=False, rank=1)

In [ ]:
plot_one_breath_id(top=False, rank=5)

In [ ]:
plot_one_breath_id(top=False, rank=10)

In [ ]:
plot_one_breath_id(top=False, rank=15)

In [ ]:
plot_one_breath_id(top=False, rank=30)

In [ ]:
plot_one_breath_id(top=False, rank=50)

In [ ]:
plot_one_breath_id(top=False, rank=77)

In [ ]:
plot_one_breath_id(top=False, rank=101)

In [ ]:
plot_one_breath_id(top=False, rank=200)

# 所感
*   スケールが大きなものほど外しやすい？
*   全体の時系列の形は合ってそうなので、無闇にconvとかattentionとかしたところで感
*   スケールが外しやすいものと特徴量の関係が知りたかったりする



In [ ]:
X_valid = X_valid[rank_mae_valid.index]

In [ ]:
X_valid.shape

In [ ]:
valid_features = pd.DataFrame(data=X_valid.reshape(-1, len(cfg['features']['all_features'])), columns=cfg['features']['all_features'])

In [ ]:
breath_id_list = rank_mae_valid['breath_id'].tolist()
breath_id_np = np.vstack([breath_id_list for _ in range(80)]).T.reshape(-1)
valid_features['breath_id'] = breath_id_np

In [ ]:
# merge
valid_df = pd.merge(rank_mae_valid, valid_features, on='breath_id')

In [ ]:
valid_df

In [ ]:
valid_df['is_huge_error'] = (valid_df['mae'] >= 0.4)

In [ ]:
valid_df.head()

In [ ]:
# スケールの違いによる
import seaborn as sns

In [ ]:
# u_inは変換後も大きく偏っている
# maeが大きいものは特に０から離れている（ガウス分布から離れている）印象
plt.figure()
plt.hist(valid_df.loc[valid_df['is_huge_error']==False, 'u_in'], color='blue', density=True, alpha=0.4)
plt.hist(valid_df.loc[valid_df['is_huge_error']==True, 'u_in'], color='orange', density=True, alpha=0.4)
# plt.hist(train['u_in'], color='green', density=True, alpha=0.4, bins=1000)
plt.xlabel('u_in')
plt.show()

In [ ]:
# u_inの最大値と予測しにくさに関係はなさそう
u_in_max_list = valid_df['u_in'].values.reshape(-1, 80).max(axis=1)
mae_list = valid_df['mae'].values.reshape(-1, 80).mean(axis=1)
plt.figure()
plt.scatter(u_in_max_list, mae_list)
plt.xlabel('max(u_in)')
plt.ylabel('mae')
plt.ylim(0,1)
plt.show()
plt.close()

In [ ]:
# Rとmaeの関係

sns.set()
sns.set_style('whitegrid')
sns.color_palette()

x1 = valid_df.loc[valid_df['R_5']==1, 'mae'].values.reshape(-1, 80)[:, 0]
x2 = valid_df.loc[valid_df['R_20']==1, 'mae'].values.reshape(-1, 80)[:, 0]
x3 = valid_df.loc[valid_df['R_50']==1, 'mae'].values.reshape(-1, 80)[:, 0]

plt.violinplot([x1, x2, x3])
plt.xticks(ticks=[1, 2, 3])
plt.xlabel(['R_5', 'R_20', 'R_50'])
plt.show()
plt.close()

In [ ]:
# Cとmaeの関係

sns.set()
sns.set_style('whitegrid')
sns.set_palette('gray')

x1 = valid_df.loc[valid_df['C_10']==1, 'mae'].values.reshape(-1, 80)[:, 0]
x2 = valid_df.loc[valid_df['C_20']==1, 'mae'].values.reshape(-1, 80)[:, 0]
x3 = valid_df.loc[valid_df['C_50']==1, 'mae'].values.reshape(-1, 80)[:, 0]

plt.violinplot([x1, x2, x3])
plt.xticks(ticks=[1, 2, 3])
plt.xlabel(['C_10', 'C_20', 'C_50'])
plt.show()
plt.close()

In [ ]:
valid_df['pressure'] = y_valid.reshape(-1)

In [ ]:
# Rとpressureの関係

sns.set()
sns.set_style('whitegrid')
sns.set_palette('gray')

x1 = valid_df.loc[valid_df['R_5']==1, 'pressure'].values.reshape(-1, 80).max(axis=1)
x2 = valid_df.loc[valid_df['R_20']==1, 'pressure'].values.reshape(-1, 80).max(axis=1)
x3 = valid_df.loc[valid_df['R_50']==1, 'pressure'].values.reshape(-1, 80).max(axis=1)

plt.violinplot([x1, x2, x3])
plt.xticks(ticks=[1, 2, 3])
plt.xlabel(['R_5', 'R_20', 'R_50'])
plt.ylabel('max(pressure)')
plt.show()
plt.close()

In [ ]:
# Cとpressureの関係

sns.set()
sns.set_style('whitegrid')
sns.set_palette('gray')

x1 = valid_df.loc[valid_df['C_10']==1, 'pressure'].values.reshape(-1, 80).max(axis=1)
x2 = valid_df.loc[valid_df['C_20']==1, 'pressure'].values.reshape(-1, 80).max(axis=1)
x3 = valid_df.loc[valid_df['C_50']==1, 'pressure'].values.reshape(-1, 80).max(axis=1)

plt.violinplot([x1, x2, x3])
plt.xticks(ticks=[1, 2, 3])
plt.xlabel(['C_10', 'C_20', 'C_50'])
plt.ylabel('max(pressure)')
plt.show()
plt.close()

In [ ]:
# R__Cとpressureの関係

sns.set()
sns.set_style('whitegrid')
sns.set_palette('gray')

x1 = valid_df.loc[valid_df['R__C_5__10']==1, 'pressure'].values.reshape(-1, 80).max(axis=1)
x2 = valid_df.loc[valid_df['R__C_5__20']==1, 'pressure'].values.reshape(-1, 80).max(axis=1)
x3 = valid_df.loc[valid_df['R__C_5__50']==1, 'pressure'].values.reshape(-1, 80).max(axis=1)
x4 = valid_df.loc[valid_df['R__C_20__10']==1, 'pressure'].values.reshape(-1, 80).max(axis=1)
x5 = valid_df.loc[valid_df['R__C_20__20']==1, 'pressure'].values.reshape(-1, 80).max(axis=1)
x6 = valid_df.loc[valid_df['R__C_20__50']==1, 'pressure'].values.reshape(-1, 80).max(axis=1)
x7 = valid_df.loc[valid_df['R__C_50__10']==1, 'pressure'].values.reshape(-1, 80).max(axis=1)
x8 = valid_df.loc[valid_df['R__C_50__20']==1, 'pressure'].values.reshape(-1, 80).max(axis=1)
x9 = valid_df.loc[valid_df['R__C_50__50']==1, 'pressure'].values.reshape(-1, 80).max(axis=1)

plt.violinplot([x1, x2, x3, x4, x5, x6, x7, x8, x9])
plt.xticks(ticks=np.arange(1, 10).tolist())
plt.xlabel(['R__C_5__10', 'R__C_5__20', 'R__C_5__50', 'R__C_20__10', 'R__C_20__20', 'R__C_20__50', 'R__C_50__10', 'R__C_50__20', 'R__C_50__50'])
plt.ylabel('max(pressure)')
plt.show()
plt.close()

In [ ]:
# R__Cとmaeの関係

sns.set()
sns.set_style('whitegrid')
sns.set_palette('gray')

x1 = valid_df.loc[valid_df['R__C_5__10']==1, 'mae'].values.reshape(-1, 80).max(axis=1)
x2 = valid_df.loc[valid_df['R__C_5__20']==1, 'mae'].values.reshape(-1, 80).max(axis=1)
x3 = valid_df.loc[valid_df['R__C_5__50']==1, 'mae'].values.reshape(-1, 80).max(axis=1)
x4 = valid_df.loc[valid_df['R__C_20__10']==1, 'mae'].values.reshape(-1, 80).max(axis=1)
x5 = valid_df.loc[valid_df['R__C_20__20']==1, 'mae'].values.reshape(-1, 80).max(axis=1)
x6 = valid_df.loc[valid_df['R__C_20__50']==1, 'mae'].values.reshape(-1, 80).max(axis=1)
x7 = valid_df.loc[valid_df['R__C_50__10']==1, 'mae'].values.reshape(-1, 80).max(axis=1)
x8 = valid_df.loc[valid_df['R__C_50__20']==1, 'mae'].values.reshape(-1, 80).max(axis=1)
x9 = valid_df.loc[valid_df['R__C_50__50']==1, 'mae'].values.reshape(-1, 80).max(axis=1)

plt.violinplot([x1, x2, x3, x4, x5, x6, x7, x8, x9])
plt.xticks(ticks=np.arange(1, 10).tolist())
plt.xlabel(['R__C_5__10', 'R__C_5__20', 'R__C_5__50', 'R__C_20__10', 'R__C_20__20', 'R__C_20__50', 'R__C_50__10', 'R__C_50__20', 'R__C_50__50'])
plt.ylabel('max(mae)')
plt.ylim(0, 1)
plt.show()
plt.close()

In [ ]:
# 

mae_np = valid_df.loc[:, 'mae'].values[::80]
pressure_np = valid_df.loc[:, 'pressure'].values.reshape(-1, 80).max(axis=1)

plt.figure()
plt.scatter(pressure_np, mae_np)
plt.xlabel('max(pressure)')
plt.ylabel('mae')
plt.show()
plt.close()

In [ ]:
np.arange(80)[::20]

# ゴミ

In [ ]:
5788160/36176

In [ ]:
train

In [ ]:
keras.layers.Bidirectional(keras.layers.LSTM(1024))(inputs)

In [ ]:
inputs

In [ ]:
y_train[~.duplicated()]

In [ ]:
pred_valid.reshape(-1, 80)[valid_mask].shape

In [ ]:
y_valid[valid_mask].shape

In [ ]:
valid_mask.shape

In [ ]:
X_train.shape

In [ ]:
!nvidia-smi

In [ ]:
train.head()